<a href="https://colab.research.google.com/github/takumitakise/LanderTest/blob/main/Lander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sympy as sy
import numpy as np

In [2]:
x1 = sy.symbols("x1")
x2 = sy.symbols("x2")
x3 = sy.symbols("x3")
x4 = sy.symbols("x4")
x5 = sy.symbols("x5")
x6 = sy.symbols("x6")
x7 = sy.symbols("x7")
x8 = sy.symbols("x8")
x9 = sy.symbols("x9")
x10 = sy.symbols("x10")
x = sy.Matrix([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10])

In [3]:
u1 = sy.symbols("u1")
u2 = sy.symbols("u2")
u3 = sy.symbols("u3")
u4 = sy.symbols("u4")
u5 = sy.symbols("u5")
u6 = sy.symbols("u6")
u = sy.Matrix([u1,u2,u3,u4,u5,u6])

In [4]:
R = sy.Matrix([[2 * x4**2 - 1 + 2 * x5**2, 2 * ( x5 * x6 + x4 * x7 ), 2 * ( x5 * x7 - x4 * x6 )],
              [2 * ( x5 * x6 - x4 * x7 ), 2 * x4**2 - 1 + 2 * x6**2, 2 * ( x6 * x7 + x4 * x5 )],
              [2 * ( x5 * x7 + x4 * x6 ), 2 * ( x6 * x7 - x4 * x5 ), 2 * x4**2 - 1 + 2 * x7**2]])

In [5]:
m = sy.symbols("m")
g = sy.symbols("g")
dx_x = sy.symbols("dx_x")
dx_y = sy.symbols("dx_y")
dx_z = sy.symbols("dx_z")
dy_x = sy.symbols("dy_x")
dy_y = sy.symbols("dy_y")
dy_z = sy.symbols("dy_z")
J1 = sy.symbols("J1")
J2 = sy.symbols("J2")
J3 = sy.symbols("J3")
M = sy.Matrix([[m, 0, 0], [0, m, 0], [0, 0, m]])
J = sy.Matrix([[J1, 0, 0], [0, J2, 0], [0, 0, J3]])

In [6]:
Location_2dot = M**(-1) * ( R * sy.Matrix( [0, 0, - m * g] ) + sy.Matrix([u1, u2, 0]) \
                            - sy.Matrix([[0, - x10, x9], [x10, 0, - x8], [- x9, x8, 0]]) * M * sy.Matrix([x1, x2, x3]) )

In [7]:
quaternion_dot = 1 / 2 * sy.Matrix([[0, - x8, - x9, - x10], [x8, 0, x10, - x9], [x9, - x10, 0, x8], [x10, x9, - x8, 0]]) \
                         * sy.Matrix([x4, x5, x6, x7])

In [8]:
omega_dot = J**(-1) * sy.Matrix([[0, - dx_z, dx_y], [dx_z, 0, - dx_x], [-dx_y, dx_x, 0]]) * sy.Matrix([u1, 0, 0])\
            + sy.Matrix([[0, - dy_z, dy_y], [dy_z, 0, - dy_x], [- dy_y, dy_x, 0]]) * sy.Matrix([0, u2, 0])\
            - sy.Matrix([[0, - x10, x9], [x10, 0, - x8], [- x9, x8, 0]]) * J * sy.Matrix([x8, x9, x10])

In [9]:
func = sy.Matrix([Location_2dot,quaternion_dot,omega_dot])

In [10]:
uxmax = sy.symbols("uxmax")
uymax = sy.symbols("uymax")
Cx = sy.Matrix([u1**2 - uxmax**2 + u3**2])
Cy = sy.Matrix([u2**2 - uymax**2 + u4**2])

In [11]:
mini = sy.Matrix([x1, x2, x3 - (x1**2 + x2**2 + x3**2)**(1/2)])

In [12]:
sf1 = sy.symbols("sf1")
sf2 = sy.symbols("sf2")
sf3 = sy.symbols("sf3")
q1 = sy.symbols("q1")
q2 = sy.symbols("q2")
q3 = sy.symbols("q3")
r1 = sy.symbols("r1")
r2 = sy.symbols("r2")
r3 = sy.symbols("r3")
r4 = sy.symbols("r4")
Sf = sy.Matrix([[sf1, 0, 0],[0, sf2, 0],[0, 0, sf3]])
Q = sy.Matrix([[q1, 0, 0],[0, q2, 0],[0, 0, q3]])
r = sy.Matrix([[r1, 0, 0, 0],[0, r2, 0, 0],[0, 0, r3, 0],[0, 0, 0, r4]])

In [13]:
Phi = 1/2 * mini.T * Sf * mini
L = 1/2 * (mini.T * Q * mini + sy.Matrix([r1 * u1**2 + r2 * u2**2])) - sy.Matrix([r3 * u3 - r4 * u4])
Phix = Phi.jacobian(x).T

In [14]:
lmd1 = sy.symbols("lmd1")
lmd2 = sy.symbols("lmd2")
lmd3 = sy.symbols("lmd3")
lmd4 = sy.symbols("lmd4")
lmd5 = sy.symbols("lmd5")
lmd6 = sy.symbols("lmd6")
lmd7 = sy.symbols("lmd7")
lmd8 = sy.symbols("lmd8")
lmd9 = sy.symbols("lmd9")
lmd10 = sy.symbols("lmd10")
Lmd = sy.Matrix([lmd1,lmd2,lmd3,lmd4,lmd5,lmd6,lmd7,lmd8,lmd9,lmd10])

In [15]:
H = L + (Lmd).T * func + u5 * Cx + u6 * Cy
Hu = H.jacobian(u).T
Hx = H.jacobian(x).T
Huu = Hu.jacobian(u).T

In [16]:
param = sy.Matrix([m,g,dx_x,dx_y,dx_z,dy_x,dy_y,dy_z,J1,J2,J3,uxmax,uymax])
contr = sy.Matrix([sf1,sf2,sf3,q1,q2,q3,r1,r2,r3,r4])

In [17]:
func_func = sy.lambdify((x,u,param), func, "numpy")
func_Phix = sy.lambdify((x,param,contr), Phix, "numpy")
func_Hu = sy.lambdify((x,u,Lmd,param,contr), Hu, "numpy")
func_Hx = sy.lambdify((x,u,Lmd,param,contr), Hx, "numpy")
func_Huu = sy.lambdify((x,u,param,contr), Huu, "numpy")

In [18]:
def GMRES(x_current,du,u,param,contr,len_x,len_u,len_lmd,T,dv,ht,zeta):
  k = len_u * dv
  dx = func_func(x_current,u[0:len_u],param)
  Fxt = CalcF(x_current+dx*ht,u,param,contr,len_x,len_u,len_lmd,T,dv)
  F = CalcF(x_current,u,param,contr,len_x,len_u,len_lmd,T,dv)
  Right = -zeta * F - ( Fxt-F ) / ht
  Fuxt = CalcF(x_current+dx*ht,u+du*ht,param,contr,len_x,len_u,len_lmd,T,dv)
  Left = ( Fuxt - Fxt ) / ht

  r0 = Right - Left
  du_new = np.zeros([k, 1])
  v = np.zeros([k, k + 1])
  r0_norm = np.linalg.norm(r0)
  v[:,[0]] = r0 / r0_norm
  h = np.zeros([k+1,k+1])
  y = np.zeros([k, 1])
  y_pre = np.zeros([k, 1])
  e = np.zeros([k+1,1])
  e[1] = 1

  for num in range(k):
    Fuxt = CalcF(x_current+dx*ht,u+v[:,[num+1]]*ht,param,contr,len_x,len_u,len_lmd,T,dv)
    Av = ( Fuxt - Fxt ) / ht
    Sum = np.zeros([k, 1])

    for num2 in range(num+1):
      h[num2,num] = np.dot(Av.T, v[:,num2])
      Sum = Sum + h[num2,num] * v[:,num2].reshape(1, -1).T
    
    v_est = Av - Sum
    h[num+1,num] = np.linalg.norm(v_est)
    v[:,[num+1]] = v_est / h[num+1,num]
    inv_h = np.linalg.pinv(h[0:num+1,0:num])
    y[0:num] =  np.dot(inv_h, r0_norm * e[:num+1])
    judge_value = r0_norm * e[:num+1] - np.dot(h[0:num+1, 0:num], y[0:num])
    if np.linalg.norm(judge_value) < 0.001:
      du_new = du + np.dot(v[:,0:num], y_pre[0:num])
      return du_new
      break

In [19]:
def Forward(x_current,u,param,len_x,len_u,T,dv):
  dt = T / dv
  x = np.zeros([len_x*(dv),1])
  x[0:len_x] = x_current
  for num in range(dv-1):
    x[len_x*(num+1):len_x*(num+2)] = x[len_x*(num+0):len_x*(num+1)] + func_func(x[len_x*(num+0):len_x*(num+1)],u[len_u*(num+0):len_u*(num+1)],param) * dt
  return x

In [20]:
def Backward(x,u,param,contr,len_x,len_u,len_lmd,T,dv):
  dt = T / dv
  lmd = np.zeros([len_lmd*(dv),1])
  lmd[len_lmd*(dv-1):len_lmd*(dv)] = func_Phix(x[len_x*(dv-1):len_x*(dv)],param,contr)
  for num in range(0, dv-1)[::-1]:
    lmd[len_lmd*(num+0):len_lmd*(num+1)] = lmd[len_lmd*(num+1):len_lmd*(num+2)] \
                                           + func_Hx(x[len_x*(num+1):len_x*(num+2)],u[len_u*(num+1):len_u*(num+2)],lmd[len_lmd*(num+1):len_lmd*(num+2)],param,contr) * dt
  return lmd

In [21]:
def CalcF(x_current,u,param,contr,len_x,len_u,len_lmd,T,dv):
  x = Forward(x_current,u,param,len_x,len_u,T,dv)
  lmd = Backward(x,u,param,contr,len_x,len_u,len_lmd,T,dv)
  F = np.zeros([len_u*(dv),1])
  for num in range(dv):
    F[len_u*(num):len_u*(num+1)] = func_Hu(x[len_x*(num):len_x*(num+1)],u[len_u*(num):len_u*(num+1)],lmd[len_lmd*(num):len_lmd*(num+1)],param,contr)
  return F

In [22]:
parapa = [50,3.71,0,0,-1,0,0,-1,1,1,4,10,10]
concon = [5,5,5,10,10,10,100,100,10,10]
len_x = 10
len_u = 6
len_lmd = len_x
dv = 10
T = 1
x_current = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]).reshape(1, -1).T
uu = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20,21, 22, 23, 24, 25, 26, 27, 28, 29, 30,31, 32, 33, 34, 35, 36, 37, 38, 39, 40,41, 42, 43, 44, 45, 46, 47, 48, 49, 50,51, 52, 53, 54, 55, 56, 57, 58, 59, 60]).reshape(1, -1).T
duu = uu
ht = 0.1
zeta = 10

In [26]:
ii = Forward(x_current,uu,parapa,len_x,len_u,T,dv)
ii

array([[ 1.00000000e+00],
       [ 2.00000000e+00],
       [ 3.00000000e+00],
       [ 4.00000000e+00],
       [ 5.00000000e+00],
       [ 6.00000000e+00],
       [ 7.00000000e+00],
       [ 8.00000000e+00],
       [ 9.00000000e+00],
       [ 1.00000000e+01],
       [-7.86000000e+00],
       [-4.26000000e+01],
       [-4.55590000e+01],
       [-4.20000000e+00],
       [ 6.45000000e+00],
       [ 8.10000000e+00],
       [ 8.85000000e+00],
       [-1.88000000e+01],
       [ 3.29000000e+01],
       [ 1.00000000e+01],
       [ 3.18450550e+01],
       [ 1.78374300e+01],
       [-2.22339575e+02],
       [-1.58865000e+01],
       [-1.10250000e-01],
       [-1.03530000e+01],
       [ 2.49742500e+01],
       [-1.16700000e+02],
       [-2.42000000e+01],
       [ 1.00000000e+01],
       [-3.64271338e+02],
       [ 2.77127395e+03],
       [-7.40932582e+02],
       [-4.15440638e+01],
       [ 1.17629820e+02],
       [-1.36799959e+02],
       [-4.32453525e+01],
       [-4.27000000e+01],
       [-3.7

In [27]:
rr = Backward(ii,uu,parapa,concon,len_x,len_u,len_lmd,T,dv)
rr

array([[-3.24930158e+49],
       [-3.46089703e+49],
       [ 1.31942453e+49],
       [-3.26784509e+61],
       [-2.09727039e+61],
       [-2.76723745e+61],
       [ 6.68086610e+61],
       [-4.47927889e+61],
       [ 7.37568373e+61],
       [ 9.66905812e+62],
       [-2.20656821e+48],
       [-9.60645677e+48],
       [-1.21255028e+49],
       [-8.32214623e+60],
       [-2.35537660e+61],
       [-2.76348303e+61],
       [-2.07509764e+60],
       [-2.05524168e+61],
       [-6.82751909e+60],
       [ 7.22054973e+62],
       [-3.27301210e+48],
       [ 3.11517401e+47],
       [-5.69405490e+47],
       [-4.84636956e+60],
       [ 2.36054440e+59],
       [ 1.71921016e+59],
       [-3.78074916e+60],
       [-6.87083641e+59],
       [-5.96956290e+60],
       [ 5.09115183e+62],
       [-2.21050102e+46],
       [-4.05682036e+46],
       [ 8.76324626e+46],
       [ 2.16643024e+57],
       [-1.68295741e+59],
       [ 2.76545459e+59],
       [-2.86473686e+58],
       [ 1.44026385e+60],
       [-5.5

In [24]:
jj = CalcF(x_current,uu,parapa,concon,len_x,len_u,len_lmd,T,dv)
jj

array([[-7.37568373e+61],
       [-4.47927889e+61],
       [ 2.00000000e+01],
       [ 5.80000000e+01],
       [-9.00000000e+01],
       [-8.00000000e+01],
       [ 6.82751909e+60],
       [-2.05524168e+61],
       [ 1.88000000e+02],
       [ 2.50000000e+02],
       [ 3.00000000e+01],
       [ 6.40000000e+01],
       [ 5.96956290e+60],
       [-6.87083641e+59],
       [ 5.00000000e+02],
       [ 5.86000000e+02],
       [ 2.94000000e+02],
       [ 3.52000000e+02],
       [ 5.56758955e+59],
       [ 1.44026385e+60],
       [ 9.56000000e+02],
       [ 1.06600000e+03],
       [ 7.02000000e+02],
       [ 7.84000000e+02],
       [-2.84061342e+59],
       [ 2.69288671e+59],
       [ 1.55600000e+03],
       [ 1.69000000e+03],
       [ 1.25400000e+03],
       [ 1.36000000e+03],
       [-7.73268927e+58],
       [-5.14561463e+58],
       [ 2.30000000e+03],
       [ 2.45800000e+03],
       [ 1.95000000e+03],
       [ 2.08000000e+03],
       [ 3.26686657e+57],
       [-1.92298355e+58],
       [ 3.1

In [23]:
ww = GMRES(x_current,duu,uu,parapa,concon,len_x,len_u,len_lmd,T,dv,ht,zeta)
ww

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


array([[ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.],
       [11.],
       [12.],
       [13.],
       [14.],
       [15.],
       [16.],
       [17.],
       [18.],
       [19.],
       [20.],
       [21.],
       [22.],
       [23.],
       [24.],
       [25.],
       [26.],
       [27.],
       [28.],
       [29.],
       [30.],
       [31.],
       [32.],
       [33.],
       [34.],
       [35.],
       [36.],
       [37.],
       [38.],
       [39.],
       [40.],
       [41.],
       [42.],
       [43.],
       [44.],
       [45.],
       [46.],
       [47.],
       [48.],
       [49.],
       [50.],
       [51.],
       [52.],
       [53.],
       [54.],
       [55.],
       [56.],
       [57.],
       [58.],
       [59.],
       [60.]])